# Setup

In [1]:
"""
Add parent directory to current path
"""
import os.path
import sys
p = os.path.abspath('..')
if p not in sys.path:
    sys.path.insert(0,p)
    
"""
Add tiger-env directory to current path
Still not sure why this is needed.
"""
p = os.path.abspath('../../custom_envs/gym-tiger')
if p not in sys.path:
    sys.path.insert(0, p)

# Simulate

In [2]:
import gym
import gym_tiger

In [3]:
env = gym.make('Tiger-v0')

/Users/jblandin/miniconda3/envs/rlpomdp/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [4]:
for i in range(25):
    obs, rew, done, info = env.step(0)
    obs = env.translate_obs(obs)
    print(obs, rew, done)

GROWL_LEFT -1 False
GROWL_LEFT -1 False
GROWL_RIGHT -1 False
GROWL_LEFT -1 False
GROWL_RIGHT -1 False
GROWL_LEFT -1 False
GROWL_RIGHT -1 False
GROWL_LEFT -1 False
GROWL_LEFT -1 False
GROWL_LEFT -1 False
GROWL_LEFT -1 False
GROWL_LEFT -1 False
GROWL_LEFT -1 False
GROWL_RIGHT -1 False
GROWL_LEFT -1 False
GROWL_LEFT -1 False
GROWL_LEFT -1 False
GROWL_LEFT -1 False
GROWL_LEFT -1 False
GROWL_LEFT -1 False
GROWL_LEFT -1 False
GROWL_RIGHT -1 False
GROWL_LEFT -1 False
GROWL_LEFT -1 False
GROWL_LEFT -1 False


In [5]:
env.step(2)

([-1, -1], 10, True, {})